In [1]:
# implementation of LRU (least recently used cache) using linked lists approach.

In [2]:
class Node:
    def __init__(self, key, value):
        self.key = key 
        self.value = value
        self.prev = None
        self.next = None

    def __repr__(self):
        return f"{self.key}:{self.value}"
        

In [16]:
class DoublyLinkedList:
    def __init__(self):
        self.head = Node(0, 'h')
        self.tail = Node(0, 't')
        self.head.next = self.tail
        self.tail.prev = self.head

    def __repr__(self):
        nodes = []
        curr = self.head.next
        while curr != self.tail:
            nodes.append(f'[{curr.key}:{curr.value}]')
            curr = curr.next
        return '<->'.join(nodes)

    def insert(self, key, value):
        node = Node(key, value)
        if self.head.next == self.tail:
            # insert after head
            self.head.next = node
            node.prev = self.head
            node.next = self.tail
            self.tail.prev = node
        
        if self.head.next != self.tail:
            # if there is a node that exists after head
            other = self.head.next
            other.next = node
            node.prev = other
            node.next = self.tail
            self.tail.prev = node
        
            

In [18]:
x = DoublyLinkedList()
x.insert(1, 'A')
x.insert(2, 'B')
x.insert(3, 'C')
x

[1:A]<->[3:C]